In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
# ignorando os warnings
import warnings
warnings.filterwarnings('ignore')
# plotando o matplotlib
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)


Importando dados / importing data

In [86]:
reembolso = pd.read_csv('Pedidos de reembolso de executivos.csv', sep=';')

In [87]:
reembolso.head()

,bugged_date,receipt_date,employee_id,work_group,state_code,employee,receipt_social_security_number,receipt_description,establishment_name,receipt_value
0,0,27/03/2013 00:00,1772,grupo 1,SP,func 1,3.530749e+12,Fuels and lubricants.,AUTO POSTO 314 NORTE LTDA,70
1,0,24/07/2013 00:00,1772,grupo 1,SP,func 1,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,104
2,0,17/02/2013 00:00,1772,grupo 1,SP,func 1,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,100
3,0,15/03/2013 00:00,1772,grupo 1,SP,func 1,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,100
4,0,27/01/2013 00:00,1772,grupo 1,SP,func 1,8.202116e+12,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,77


In [88]:
reembolso.shape

(1048575, 10)

In [105]:
#reduzindo o tamanho da base para rodar o LOF / reducing the sample size to run the LOF
reembolso = reembolso[:10000]

In [106]:
reembolso.shape

(10000, 14)

In [107]:
reembolso.drop(reembolso[reembolso['receipt_date'] == '2013'].index, inplace = True) 
reembolso.drop(reembolso[reembolso['receipt_date'] == '2014'].index, inplace = True) 
reembolso.drop(reembolso[reembolso['receipt_date'] == '2011'].index, inplace = True) 
reembolso = reembolso.assign(day=reembolso['receipt_date'].str[:2], month=reembolso['receipt_date'].str[3:5], year=reembolso['receipt_date'].str[6:10])

In [108]:
X = reembolso.drop(['bugged_date', 'employee', 'receipt_social_security_number', 'work_group','state_code', 'receipt_date'], axis=1)

In [109]:
X.head()

,employee_id,receipt_description,establishment_name,receipt_value,day,month,year,anomaly_by_lof
0,1772,Fuels and lubricants.,AUTO POSTO 314 NORTE LTDA,70,27,03,2013,1
1,1772,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,104,24,07,2013,1
2,1772,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,100,17,02,2013,1
3,1772,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,100,15,03,2013,1
4,1772,Fuels and lubricants.,AUTO POSTO AEROPORTO LTDA,77,27,01,2013,-1


In [110]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [111]:
X['receipt_description'] = X['receipt_description'].astype('category')
#X['establishment_name'] = X['establishment_name'].astype('category')

X['day'] = X['day'].astype('int64')
X['month'] = X['month'].astype('int64')
X['year'] = X['year'].astype('int64')

In [112]:
X.drop(['establishment_name'], axis=1, inplace=True)

In [113]:
X.head()

,employee_id,receipt_description,receipt_value,day,month,year,anomaly_by_lof
0,1772,Fuels and lubricants.,70,27,3,2013,1
1,1772,Fuels and lubricants.,104,24,7,2013,1
2,1772,Fuels and lubricants.,100,17,2,2013,1
3,1772,Fuels and lubricants.,100,15,3,2013,1
4,1772,Fuels and lubricants.,77,27,1,2013,-1


In [114]:
# preprocess = make_column_transformer((['receipt_date','employee_id','work_group','state_code','receipt_description','establishment_name'], OneHotEncoder()))
#columnTransformer = make_column_transformer((OneHotEncoder(), ['employee_id','receipt_description','establishment_name']), remainder='passthrough')

columnTransformer = make_column_transformer((OneHotEncoder(), ['employee_id','receipt_description']), remainder='passthrough')

In [115]:
X_trans = columnTransformer.fit_transform(X)

In [116]:
from sklearn.neighbors import LocalOutlierFactor

In [117]:
clf = LocalOutlierFactor(n_neighbors=50, contamination=0.1, p=1)  #p = 1, this is equivalent to using manhattan_distance (l1)

In [118]:
y_pred = clf.fit_predict(X_trans)

In [119]:
reembolso['anomaly_by_lof'] = y_pred
outliers = reembolso.loc[reembolso['anomaly_by_lof']==-1]
outlier_index = list(outliers.index)

In [120]:
reembolso['anomaly_by_lof'].value_counts(normalize=True)*100

 1    90.0
-1    10.0
Name: anomaly_by_lof, dtype: float64